In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import glob

model = YOLO("./models/yolo11n-detect-custom.pt")
model_classify = YOLO("./models/yolo11n-classify-custom.pt")

image_files = glob.glob("./predict/image_*")

filtered_boxes = []

def resize_to_square(image, size=640):
    h, w = image.shape[:2]
    scale = size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    square_image = np.zeros((size, size, 3), dtype=np.uint8)

    x_offset = (size - new_w) // 2
    y_offset = (size - new_h) // 2
    square_image[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    return square_image

def get_precise_box(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, binary = cv2.threshold(image, 170, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5,5), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("Object not found")
        return

    main_contour = max(contours, key=cv2.contourArea)

    return main_contour

def iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

for file in image_files:
    image = cv2.imread(file)
    results = model(image, conf=0.6, agnostic_nms=True, iou=0.3)

    image_filtered = image.copy()

    class_names = {0: "Plate1", 1: "Plate3"}
    class_colors = {0: (255, 0, 0), 1: (0, 0, 255)}

    res = results[0].boxes

    detected_boxes = []
    for box in res:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        class_id = int(box.cls.cpu())
        confidence = box.conf.cpu().numpy()[0]

        detected_boxes.append((x1, y1, x2, y2))
        color = class_colors.get(class_id, (0, 255, 0))
        cv2.rectangle(image_filtered, (x1, y1), (x2, y2), color, 1)

        class_name = class_names.get(class_id, "Unknown")
        confidence_text = f"({confidence:.2f})"

        cv2.putText(image_filtered, confidence_text, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 1)

        object_image = resize_to_square(image[y1:y2,x1:x2])

        border_w = 15
        precise_box = get_precise_box(image[y1-border_w:y2+border_w, x1-border_w:x2+border_w])
        cv2.drawContours(image_filtered, [precise_box+[x1-border_w, y1-border_w]], -1, (0, 255, 0), 2)

        classify_results = model_classify(object_image)

        if classify_results and hasattr(classify_results[0], 'probs'):
            predicted_class_name = f"{class_names[classify_results[0].probs.top1]} ({classify_results[0].probs.top1conf.cpu().numpy():.2f})"
        else:
            predicted_class_name = "Unknown"

        cv2.putText(image_filtered, predicted_class_name, (x1, y2 + 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 1)

    plt.figure()
    plt.figure(figsize=(40,10))
    plt.imshow(cv2.cvtColor(image_filtered, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()